# Influence Maximisation Problem with time and random delays

This is the issue of maximising value in a netowork by selecting nodes that are most likely to yield the highest value. In this notebook we will implement some well studied models and extend them to account for time delays. We will design heuristic and greedy strategies to tackle the influence maximisation problem and analyse performance of the algorithms.

## Jargon: understanding vocabulary

Here we put some vocabulary we might need in order to understand and tackle the problem.

* **States** is how we represent the node, 0 is inactive, -1 latent active, and 1 is active.
* **Propogation value** PV(S) is the expected value of activated nodes in a seed set, S.
* **Seed set** is the set of nodes that contain the solution.
* **k seeds** is the number of initial set of nodes before starting influence maximisation.
* **Freashness function** is a function to determine the value of a node with respect to the time till activation.
* **Freashness value** is the actual value of the node.


## Generate a graph

A model where the sum of influence will determine whether a node becomes active. The influence must exceed the threshold of a node. Each edge and node has a random value assigned to it between 0 and 1 when a new instance is generated.

In [1]:
import snap
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import random
import time
import itertools
import urllib
import csv
import functools
import numpy as np
import heapq as h
import re
from statistics import mean, stdev

gen_graph uses seeds. A seed is a feature of python that allows us to generate random values associated to nodes and edges when we need them. If we want to reference a graph generated in the past we just input the seed that was used.

In [2]:
def gen_graph(size, seed=42):
    """generates a network, each value assigned to the node and edge is random float between 0 and 1"""
    random.seed(seed*size)
    G = snap.GenRndGnm(snap.PNEANet, size, random.randrange(size*2, (size*(size - 1)/2)/2))
    
    # define float and str attributes on nodes
    G.AddFltAttrN("NValFlt", 0.0)
    G.AddStrAttrN("NValStr", "0")

    # define an int attribute on edges
    G.AddFltAttrE("EValFlt", 0)

    # add attribute values, node ID for nodes, edge ID for edges
    for NI in G.Nodes():
        nid = NI.GetId()
        val = nid
        G.AddFltAttrDatN(nid, random.random(), "NValFlt")
        G.AddStrAttrDatN(nid, str(val), "NValStr")
        G.AddIntAttrDatN(nid, 0, "State")

        for nid1 in NI.GetOutEdges():
            eid = G.GetEId(nid,nid1)
            val = eid
            G.AddFltAttrDatE(eid, random.random(), "EValFlt")

    # print out attribute values
#     for n in G.Nodes():
#         nid = n.GetId()
#         fval = G.GetFltAttrDatN(nid, "NValFlt")
#         sval = G.GetStrAttrDatN(nid, "NValStr")
        
#         print("node %d, NValFlt %.2f, NValStr %s" % (nid, fval, sval))

#         for nid2 in n.GetOutEdges():
#             eid = G.GetEId(nid, nid2)
#             val1 = G.GetFltAttrDatE(eid, "EValFlt")
#             print("edge %d (%d,%d), EValFlt %f" % (eid, nid, nid2, val1))
            
    return G

In [3]:
def node_states(g, attr, size):
    """A method iterating through the network using an iterator to check the states of each node."""
    iterator = g.BegNI()
    print("NId %d has the following state %d" % (iterator.GetId() ,g.GetIntAttrDatN(iterator.GetId(), attr)))
    for i in range(0, size):
        temp = iterator.Next()
        if temp != g.EndNI():
            NId = temp.GetId()
            print("NId %d has the following state %d" % (NId ,g.GetIntAttrDatN(NId, attr)))
        else:
            break
        

In [ ]:
graph1 = gen_graph(10, 14)
node_states(graph1, "State", graph1.GetNodes())

## Linear Threshold

This is the case of where the ff(t) = 1. The freshness function can be any positive decreasing function for example, exponential, polynomial or piecewise linear function. fval(v)=ff(v.acttime) the value of a node is calculated by input of the activation time in the freshness function.

In [4]:
def edit_nodestate(graph, NId, attr, val):
    """changes the state of a specific node in the network"""
    graph.DelAttrDatN(NId, attr)
    graph.AddIntAttrDatN(NId, val, attr)
    
    return graph

In [5]:
def lt_prop(g, k):
    """simple propogation in a linear threshold way with k = 1 and ff(t) = 1"""
    s = list()
    
    #check to see if propogation already carried out
    for n in g.Nodes():
        if g.GetIntAttrDatN(n.GetId(), "State") == 1:
            return print("Influence propogation has been carried out, please re-generate graph. \n")
    
    # select k random nodes
    for i in range(0,k):
        initial = g.GetRndNId()
        while initial in s:
            initial = g.GetRndNId()
        g = edit_nodestate(g, initial, "State", 1)
        s.append(initial)
    
    # adds nodes to s that exceed thresholds in lt
    sizeS = len(s)
    while True:
        for n in g.Nodes():
            nid = n.GetId()
            if g.GetIntAttrDatN(nid, "State") == 0:
                theta = g.GetFltAttrDatN(nid, "NValFlt")
                influence = 0
                for nid2 in n.GetOutEdges():
                    if g.GetIntAttrDatN(nid2, "State") != 0:
                        eid = g.GetEId(nid, nid2)
                        influence += g.GetFltAttrDatE(eid, "EValFlt")

                if influence >= theta:
                    edit_nodestate(g, nid, "State", 1)
                    s.append(nid)

        # if no further changes to number of nodes close
        if sizeS != len(s) and len(s) <= g.GetNodes():
            sizeS = len(s)
        else:
            break
    
    # check the proportion of nodes that are activated
    print("Proportion of nodes influenced %0.0f%% \n" % ((sizeS/g.GetNodes())*100))

In [ ]:
graph2 = gen_graph(100, 10)

In [ ]:
lt_prop(graph2, 1)
node_states(graph2, "State", graph2.GetNodes())

## Independent Cascade

In [6]:
def ic_prop(g, k):
    """simple propogation in an independent casecade way with k = 1 and ff(t) = 1"""
    s = list()
    
    #check to see if propogation already carried out
    for n in g.Nodes():
        if g.GetIntAttrDatN(n.GetId(), "State") == 1:
            return print("Influence propogation has been carried out, please re-generate graph. \n")
        
    # select k random nodes
    for i in range(0,k):
        initial = g.GetRndNId()
        while initial in s:
            initial = g.GetRndNId()
        g = edit_nodestate(g, initial, "State", 1)
        s.append(initial)
    
    # start propogation, add a value to check if the node has tried to activate its neighbours(independent)
    sizeS = len(s)
    while True:
        for n in g.Nodes():
            nid = n.GetId()
            if g.GetIntAttrDatN(nid, "State") == 1:
                for nid2 in n.GetOutEdges():
                    eid = g.GetEId(nid, nid2)
                    theta = g.GetFltAttrDatN(nid2, "NValFlt")
                    if g.GetFltAttrDatE(eid, "EValFlt") >= theta and nid2 not in s:
                        edit_nodestate(g, nid2, "State", 1)
                        s.append(nid2)
        
        if sizeS != len(s) and len(s) <= g.GetNodes():
            sizeS = len(s)
        else:
            break
    
    # check the proportion of nodes that are activated
    print("Proportion of nodes influenced %0.0f%% \n" % ((sizeS/g.GetNodes())*100))                 

In [ ]:
graph3 = gen_graph(200, 12)

In [ ]:
ic_prop(graph3, 1)
node_states(graph3, "State", graph3.GetNodes())

## Time sensitive influence maximisation

We need to incorporate more information into the network in order to account for time. Firstly, each node will have three states, for 0 for inactive, 1 for latent , and 2 for active state. Once a node has been triggered to be active it goes into a latent active state, and will activate in (t+d) steps.

In [276]:
def gen_ts_graph(size, seed=42, pr=1):
    """creates a network with time delay on edges, and the capability to add three states."""
    random.seed(seed*size)
    G = snap.GenRndGnm(snap.PNEANet, size, random.randrange(size*2, (size*(size - 1)/4)))
    
    # define float and str attributes on nodes
    G.AddFltAttrN("NValFlt", 0.0)
    G.AddStrAttrN("NValStr", "0")
    
    # define an flt attribute on edges
    G.AddFltAttrE("EValFlt", 0)

    # add attribute values, node ID for nodes, edge ID for edges
    for NI in G.Nodes():
        nid = NI.GetId()
        val = nid
        G.AddFltAttrDatN(nid, random.random(), "NValFlt")
        G.AddStrAttrDatN(nid, str(val), "NValStr")
        G.AddIntAttrDatN(nid, 0, "State")
        G.AddIntAttrDatN(nid, 0, "activationTime")
        G.AddIntAttrDatN(nid, random.randrange(0, 20), "delay")

        for nid1 in NI.GetOutEdges():
            eid = G.GetEId(nid,nid1)
            val = eid
            G.AddFltAttrDatE(eid, random.random(), "EValFlt")

    if pr:
        print("Graph successfully generated! \n")
            
    return G

In [ ]:
graph4 = gen_ts_graph(8000, 20)

## Delayed Linear Threshold

The solution to the influence maximisation problem will be deterministic for the Linear threshold model. If we start at the same initial node it will give us the same solution each time. We can do a greedy or approximate approach to solve the problem for linear threshold. In the greedy approach we need to evaluate the value of a node when we add it to the solution, we determine the value using the freshness function.

In the greedy algorithm, we find k initial nodes. Once we have k nodes in our set, we start propogation; propogation ends if there are no more nodes in the latent activate state and the set of nodes activated in the **current step** is empty. If the propogation value is 0 at step t then we do not need to continue propogation. 




In [274]:
def exponential(x):
    if x >= 0:
        return (np.exp(1)**-(x*0.2))
    else:
        return print("Enter a value greater than or equal to 0")

def polynomial(x):
    if x >= 1:
        return 1/x
    else:
        return print("Enter a value greater than or equal to 1")
    
def piecewiselinear(x):
    if x >= 5 and x <= 30:
        return -x + 30
    elif x > 30 and x <= 40:
        return -0.5 * x + 20
    else:
        return print("Enter a value between 5 and 30")

In [278]:
import time

def im_g_lt(g, k, seed=42):
    # (activation time, latent activation, {node id, threshold})
    random.seed(seed*k)
    
    #check to see if k is larger than g
    if k >= g.GetNodes():
        return print("Pick a k value less than %d" % (g.GetNodes()))
    
    # nodes in latent active state 'aCurr' and nodes activated 'aT'  
    aCurr = []
    aT = []
    h.heapify(aCurr)
    h.heapify(aT)
    
    # select 1 initial node
    initial = random.randrange(0, g.GetNodes())
    h.heappush(aT, (0, 0, initial))
    
    # add initial nodes to aCurr
    initialNode = g.GetNI(initial)
    for n in initialNode.GetOutEdges():
        str1 = ''.join(str(e) for e in aCurr)
        if not re.findall("\(\d+\,\s\d+\,\s" + str(n) + "\)", str1):        
            eid = g.GetEId(initial, n)
            threshold = g.GetFltAttrDatN(n, "NValFlt")
            if g.GetFltAttrDatE(eid, "EValFlt") > threshold:
                h.heappush(aCurr, (g.GetIntAttrDatN(n, "delay"), 0, n))
        
    # Select k nodes
    timeStep = 0
    candidates = []
    h.heapify(candidates)
    
    
    for i in range(0, k - 1):
        # put things in latent active state: BFS
        queue = []
        queue.append(aT[0])
        lockTime = False
        
        while aCurr:
            timeStep += 1
            
            while queue:
                if lockTime:
                    break
                
                str1 = ''.join(str(e) for e in aCurr)
                str2 = ''.join(str(e) for e in aT)
                v = queue.pop()
                node = g.GetNI(v[2])
                for nid2 in node.GetOutEdges():
                    if re.findall("\(\d+\,\s\d+\,\s" + str(nid2) + "\)", str2):
                        queue.append((g.GetIntAttrDatN(nid2, "delay") + timeStep ,timeStep ,nid2))
                    elif not re.findall("\(\d+\,\s\d+\,\s" + str(nid2) + "\)", str1):
                        threshold = g.GetFltAttrDatN(nid2, "NValFlt")
                        influence = 0
                        for nid3 in g.GetNI(nid2).GetOutEdges():
                            eid = g.GetEId(nid2, nid3)
                            if re.findall("\(\d+\,\s\d+\,\s" + str(nid3) + "\)", str2):
                                influence += g.GetFltAttrDatE(eid, "EValFlt")
                                if influence >= threshold:
                                    h.heappush(aCurr, (g.GetIntAttrDatN(nid2, "delay") + timeStep, timeStep, nid2))
                                    break
                lockTime = True
            
            # get candidates that have activated from latent state
            if aCurr and aCurr[0][0] <= timeStep:
                while True:
                    if aCurr and aCurr[0][0] <= timeStep:
                        tempVar = h.heappop(aCurr)
                        h.heappush(candidates, tempVar)
                        lockTime = False
                    else:
                        break
                break
            else:
                if candidates:
                    break
                    
        # the largest pv will have the smallest activation time assuming exp function
        if candidates:
            node = h.heappop(candidates)
            h.heappush(aT, node)
    
    return aT

prop_lt is a method that activates the initial set of nodes and propogates influence; the freshness function is returned after propogating has completed.

In [267]:
def prop_lt(g, initialSet):
    
    # nodes in latent active state 'aCurr' and nodes activated 'aT'  
    aCurr = []
    aT = []
    h.heapify(aCurr)
    h.heapify(aT)
    
    #activate the initial nodes
    for item in initialSet:
        h.heappush(aT, item)
    
    # start propogation again with the set of nodes activated
    pV = 0
    activatedNode = False
    timeStep = 0
    
    # BFS
    lockTime = False
    iterationFirst = True

    while aCurr or iterationFirst:
        timeStep += 1
        
        # BFS starts
        queue = []
        queue.append(aT[0])
        visited = []
        
        if not lockTime:
            str1 = ''.join(str(e) for e in aCurr)
            str2 = ''.join(str(e) for e in aT)
            v = queue.pop()
            node = g.GetNI(v[2])
            for nid2 in node.GetOutEdges():
                
                cond1 = re.findall("\(\d+\,\s\d+\,\s" + str(nid2) + "\)", str2)
                cond2 = re.findall("\(\d+\,\s\d+\,\s" + str(nid2) + "\)", str1)
                
                if not nid2 in visited:
                    if len(cond1):
                        queue.append((g.GetIntAttrDatN(nid2, "delay") + timeStep ,timeStep ,nid2))
                    elif not len(cond1) and not len(cond2):
                        threshold = g.GetFltAttrDatN(nid2, "NValFlt")
                        influence = 0             
                        for nid3 in g.GetNI(nid2).GetOutEdges():
                            eid = g.GetEId(nid2, nid3)
                            if re.findall("\(\d+\,\s\d+\,\s" + str(nid3) + "\)", str2):
                                influence += g.GetFltAttrDatE(eid, "EValFlt")
                                if influence >= threshold:
                                    h.heappush(aCurr, (g.GetIntAttrDatN(nid2, "delay") + timeStep, timeStep, nid2))
                                    visited.append(nid2)
                                    break
            visited.append(v[2])
        # BFS ends
        
        lockTime = True
        iterationFirst = False
        # get candidates that have activated from latent state
        if aCurr and aCurr[0][0] <= timeStep:
            while True:
                if aCurr and aCurr[0][0] <= timeStep:
                    tempVar = h.heappop(aCurr)
                    h.heappush(aT, tempVar)
                    lockTime = False

                    # calculate the propogation value
                    freshVal = exponential(tempVar[0])
                    pV += freshVal
                else:
                    break
    
    print("Freshness value %f, number of nodes: %d" % (pV, len(aT)))

In [279]:
graph5 = gen_ts_graph(1000, 20)

Graph successfully generated! 



In [259]:
initialNodes = im_g_lt(graph5, 10, 10)

In [268]:
prop_lt(graph5, initialNodes)

Freshness value 0.315284, number of nodes: 121


In [280]:
kNodes = 0
for i in range(0, 7):
    initialNodes = im_g_lt(graph5, 5 + kNodes, 3)
    prop_lt(graph5, initialNodes)
    print("IM with k = %d nodes \n" % (kNodes + 5))
    print("--------------------")
    kNodes += 5

Freshness value 8.643720, number of nodes: 110
IM with k = 5 nodes 

--------------------


KeyboardInterrupt: 

Freshness functions: exponential, polynomial or piecewise linear.

In the approximate algorithms we look at which nodes to pick that will give use the best solution. Whereas in the greedy it does not matter, however on average the hill climbing strategy is 63% better. For example, if we select nodes with the highest degree, we will always look to add nodes with the highest degree.

In [ ]:
def im_a_lt(g, k)

## Delayed Independend Cascade